In [0]:
from pyspark.sql.functions import col,lit,current_timestamp

In [0]:
customers_df = (
       spark.readStream 
            .format("cloudFiles")
            .option("cloudFiles.format", "json")
            .option("cloudFiles.schemaLocation", "/Volumes/gizmobox/landing/operational/customers_autoloader/_schema")
            .option("cloudFiles.inferColumnTypes", "true")
            .option("cloudFiles.schemaHints","created_timestamp TIMESTAMP,date_of_birth DATE,member_since DATE")
            .option("cloudFiles.schemaEvolutionMode","rescue")
            .load("/Volumes/gizmobox/landing/operational/customers_autoloader/")
    )

In [0]:
customers_df_new_columns = customers_df.withColumn("filepath",col("_metadata.file_path")) \
                   .withColumn("ingestiondate",current_timestamp())

In [0]:
stream_query = (
                customers_df_new_columns.writeStream 
                             .format("delta") 
                             .option("checkpointLocation", "/Volumes/gizmobox/landing/operational/customers_autoloader/_checkpointsstream")
                             .option("mergeSchema","true")
                             .toTable("gizmobox.bronze.customers_autoloader")
                )

In [0]:
stream_query.stop()

In [0]:
%sql
drop table gizmobox.bronze.customers_autoloader

In [0]:
%sql
select * from gizmobox.bronze.customers_autoloader